# Multiclass Classification

While binary classifiers are used to distinguish between two classes (e.g. detect if a transaction is a fraudulent one, classify an email into either spam or non-spam and etc.), multiclass classifiers distinguish between more than two classes. 

There are various ways that we can use to perform multiclass classification by leveraging any binary classifiers. In this exercise, you will implement two such strategies for multiclass classification: _One-versus-all_ strategy and _One-versus-one_ strategy.

- **One-versus-all (OvA)**: In this strategy, we train a single binary classifier per class, with the samples of that class as positive samples and all other samples as negatives. During inference, we get the prediction from each classifier and select the class with the highest score. This strategy is also called the one-versus-the-rest strategy. 

- **One-versus-one (OvO)**: In this strategy, we train a binary classifier for every pair of classes. If there are N classes in the problem, you need to train N * (N-1) / 2 classifiers. During inference, we have to run through all N * (N-1) / 2 classifiers and ses which class wins the most votes. The main advantage of OvO strategy is that each binary classifier only needs to be train on the part of the training dataset for the two classes that it needs to separate. 

In [ ]:
# import packages
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import datasets
from sklearn.linear_model import LogisticRegression

# make this notebook's output stable across runs
np.random.seed(0)

## Avila Dataset

In this lab assignment, we use the [Avila](https://archive.ics.uci.edu/ml/datasets/Avila) data set, which has been extracted from 800 images of the the "Avila Bible," a giant Latin copy of the whole Bible produced during the XII century between Italy and Spain.  
The palaeographic analysis of the  manuscript has individuated the presence of 12 copyists. The pages written by each copyist are not equally numerous. 
Each pattern contains 10 features and corresponds to a group of 4 consecutive rows.

The prediction task consists in associating each pattern to one of the 12 copyists (labeled as: A, B, C, D, E, F, G, H, I, W, X, Y).
The data have has been normalized, by using the Z-normalization method, and divided in two data sets: a training set containing 10430 samples, and a test set  containing the 10437 samples.


In [ ]:
# Load train and test data from CSV files.
train = pd.read_csv("avila-tr.txt", header=None)
test = pd.read_csv("avila-ts.txt", header=None)

x_train = train.iloc[:,:-1]
y_train = train.iloc[:,-1]

x_test = test.iloc[:,:-1]
y_test = test.iloc[:,-1]

In [ ]:
train_counts = y_train.value_counts()
test_counts = y_test.value_counts()
print('Train class counts:
', train_counts)
print('Test class counts:
', test_counts)


Question 1.1: Check for missing Data

In [ ]:
print('Missing values in train set:', train.isnull().sum().sum())
print('Missing values in test set:', test.isnull().sum().sum())


Question 1.2: Apply Z-normalization to data

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

x_train_z = scaler.fit_transform(x_train)
x_test_z = scaler.transform(x_test)


Question 2.1: Write a method to train multiple logistic regression models performing One vs All (OvA) classification. The method allows you to pass in training features, and target. The method returns a list of models and their associated labels. 
Within the method:
- Determine the list of classes
- Create a place to store all the models
- For each class, train a model with the target variable set to 1 and 0 for all other classes
- Return the list of models trained and associated labels.

In [ ]:
def trainOvA(x, y):
    """
    Train the multiclass classifier using OvA strategy.
    """
    labels = sorted(y.unique())
    n_labels = len(labels)
    print(f'number of classes is {n_labels}')

    models = [None]*n_labels
    model_labels = [None]*n_labels
    for i, label in enumerate(labels):
        print(f'Train Logistic Regression model for class {label}')
        lr = LogisticRegression(solver='liblinear')
        binary_y = (y==label).astype(int)
        lr.fit(x, binary_y)
        models[i] = lr
        model_labels[i] = label
    return models, model_labels


Question 2.2: Write a method that leverages the multiple models trained for OvA, and outputs the majority class.

In [ ]:
def predictOvA(models, labels, x):
    """
    Make predictions on multiclass problems using the OvA strategy.
    """
    if models is None:
        sys.exit('The model has not been trained yet. Please call train() first. Exiting...')

    predictions = pd.DataFrame()
    for model, label in zip(models, labels):
        prob = model.predict_proba(x)[:,1]
        predictions[label] = prob

    return predictions.idxmax(axis=1).values


Question 2.3: Train OvA model on the Avila dataset

In [ ]:
from sklearn.linear_model import LogisticRegression
models_ova, labels_ova = trainOvA(x_train_z, y_train)


Question 2.4: Predict and evalutate the results of your model

In [ ]:
te_z_ova = predictOvA(models_ova, labels_ova, x_test_z)


In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

ova_accuracy = accuracy_score(y_test, te_z_ova)
ova_confuction_matrix = confusion_matrix(y_test, te_z_ova)

print('Accuracy of OvA classifier is {}.'.format(ova_accuracy))
print('Confusion matrix of OvA classifier: 
 {}'
      .format(ova_confuction_matrix))


---

Question 3.1: Develop a method that trains a list of models based on the OvO strategy for multiclass classification using logistic regression. 

In [ ]:
def trainOvO(x, y):
    """
    Train the multiclass classifier using OvO strategy.
    """
    labels = sorted(y.unique())
    n_labels = len(labels)
    n_models = int(n_labels * (n_labels - 1) / 2)
    print(f'number of classes is {n_labels}')

    models = [None]*n_models
    model_labels = [None]*n_models
    model_idx = 0
    for i in range(n_labels):
        for j in range(i+1, n_labels):
            label_i = labels[i]
            label_j = labels[j]
            print(f'Train Logistic Regression model to distinguish {label_i} and {label_j}')

            train_mask = y.isin([label_i, label_j])
            train_x = x[train_mask]
            train_y = y[train_mask].map({label_i:0, label_j:1})

            lr = LogisticRegression(solver='liblinear')
            lr.fit(train_x, train_y)
            models[model_idx] = lr
            model_labels[model_idx] = (label_i, label_j)
            model_idx += 1
    return models, model_labels


Question 3.2: Write a method that leverages the multiple models trained for OvO, and outputs the majority class.

In [ ]:
def predictOvO(models, labels, x):
    """
    Make predictions on multiclass problems using the OvO strategy.
    """
    if models is None:
        sys.exit('The model has not been trained yet. Please call train() first. Exiting...')

    preds = pd.DataFrame(index=range(x.shape[0]))
    for idx, (model, (label_i, label_j)) in enumerate(zip(models, labels)):
        pred = model.predict(x)
        pred_labels = np.where(pred==0, label_i, label_j)
        preds[idx] = pred_labels

    return preds.mode(axis=1).iloc[:, 0].values


Question 3.3: Train OvO model on the Avila dataset

In [ ]:
models_ovo, labels_ovo = trainOvO(x_train_z, y_train)


Question 3.4: Predict and evaluate the results of your model

In [ ]:
te_z_ovo = predictOvO(models_ovo, labels_ovo, x_test_z)


In [ ]:
ovo_accuracy = accuracy_score(y_test, te_z_ovo)
ovo_confuction_matrix = confusion_matrix(y_test, te_z_ovo)

print('Accuracy of OvO classifier is {}.'.format(ovo_accuracy))
print('Confusion matrix of OvO classifier: 
 {}'
      .format(ovo_confuction_matrix))


Question 4.1: [LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) sklearn supports two approaches for solving multi-class problems: 'ovr', 'multinomial'. Try out both approaches, and evaluate and compare the performance against what you developed in questions 2 and 3.

In [ ]:
clf = LogisticRegression(multi_class='ovr', solver='liblinear')
clf.fit(x_train_z, y_train)
y_ovr = clf.predict(x_test_z)

ovr_accuracy = accuracy_score(y_test, y_ovr)
ovr_confuction_matrix = confusion_matrix(y_test, y_ovr)

print('Accuracy of OvR classifier is {}.'.format(ovr_accuracy))
print('Confusion matrix of OvR classifier: 
 {}'
      .format(ovr_confuction_matrix))


In [ ]:
#class = multinomial
clf = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000)
clf.fit(x_train_z, y_train)
y_multinomial = clf.predict(x_test_z)

multinomial_accuracy = accuracy_score(y_test, y_multinomial)
multinomial_confuction_matrix = confusion_matrix(y_test, y_multinomial)

print('Accuracy of multinomial classifier is {}.'.format(multinomial_accuracy))
print('Confusion matrix of multinomial classifier: 
 {}'
      .format(multinomial_confuction_matrix))


Question 4: Create a new text cell in your Notebook: Complete a 50-100 word summary (or short description of your thinking in applying this week's learning to the solution) of your experience in this assignment. Include: What was your incoming experience with this model, if any? what steps you took, what obstacles you encountered. how you link this exercise to real-world, machine learning problem-solving. (What steps were missing? What else do you need to learn?) This summary allows your instructor to know how you are doing and allot points for your effort in thinking and planning, and making connections to real-world work.

My experience with this assignment involved applying logistic regression in both one-vs-all and one-vs-one settings. I began by inspecting the data and normalizing the features. Implementing the OvA and OvO helpers reinforced how binary classifiers can extend to multiclass problems. After training and evaluating the models, I compared them with scikit-learn's built-in multiclass options. This exercise highlighted the importance of preprocessing and careful evaluation when tackling real-world classification tasks.